In [10]:
import geopandas as gpd
import pandas as pd
import requests
import numpy as np

In [3]:
routes = pd.read_excel('data/routes_for_dm.xlsx')

In [4]:
routes.head()

,Unnamed: 0,from_id,to_id,slat,slon,elat,elon
0,1,13,14,34.051194,-118.92796,34.076973,-118.927960
1,2,13,37,34.051194,-118.92796,34.038302,-118.901011
2,3,13,38,34.051194,-118.92796,34.064085,-118.901011
3,4,13,61,34.051194,-118.92796,34.025408,-118.874062
4,5,13,62,34.051194,-118.92796,34.051194,-118.874062


In [7]:
http://192.168.0.116:5000/route/v1/driving/-118.92796,34.051194;-118.927960,34.076973?overview=full&geometries=geojson

1973.16

In [13]:
print(urls[0])

http://192.168.0.116:5000/route/v1/driving/-118.9279604645834,34.05119446036709;-118.9279604645834,34.07697343102303?overview=full&geometries=geojson


In [11]:
url = "http://192.168.0.116:5000/route/v1/driving/{slon},{slat};{elon},{elat}?overview=full&geometries=geojson"

blocks = np.array_split(routes, 200)

for i in [0, 1]:#range(len(blocks)):
    
    block = blocks[i]
    urls = block.apply(lambda r: url.format(
        slon=r['slon'],
        slat=r['slat'], 
        elon=r['elon'], 
        elat=r['elat']), axis=1)
    break
    rs = (grequests.get(u) for u in urls)
    responses = grequests.map(rs)
    
    routes = []
    mask = []
    
    for res in responses:
        try:
            routes.append(LineString(res.json()['routes'][0]['geometry']['coordinates']))
            mask.append(True)
        except:
            rejected.append(res)
            mask.append(False)
    
    valids = block.loc[mask]    
    routes = (
        gpd.GeoDataFrame(valids, geometry=routes, crs='EPSG:4326')
            .rename(columns={'geometry': 'route'})
            .set_geometry('route')
     )
    routes['length'] = routes.to_crs('EPSG:3857').geometry.length
    routes['from_id', 'to_id', 'lenght'].to_excel(f'data/vector/dm/{i}.geojson', driver='GeoJSON')

In [ ]:


rejected = []
tot_start = time.time()
step = 5000
last = 0
start_date, end_date = df.pickup_datetime.min(), df.pickup_datetime.max() #+ datetime.timedelta(days=7)
period = df[(df.pickup_datetime>= start_date) & (df.pickup_datetime<end_date)].reset_index(drop=True)
print('Total routes to construct:', len(period))
for new in period.index[::step]:
    start = time.time()
    block = period.loc[last:new]
    urls = block.apply(lambda r: url.format(
        slng=r['pickup_longitude'],
        slat=r['pickup_latitude'], 
        elng=r['dropoff_longitude'], 
        elat=r['dropoff_latitude']), axis=1)
    
    rs = (grequests.get(u) for u in urls)
    responses = grequests.map(rs)
    routes = []
    mask = []
    for res in responses:
        try:
            routes.append(LineString(res.json()['routes'][0]['geometry']['coordinates']))
            mask.append(True)
        except:
            rejected.append(res)
            mask.append(False)
    
    valids = block.loc[mask]    
    routes = (gpd.GeoDataFrame(valids, geometry=routes, crs='EPSG:4326')
        .rename(columns={'geometry': 'route'})
        .set_geometry('route')
        .drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])
     )
    routes['startp'] = gpd.points_from_xy(valids.pickup_longitude, valids.pickup_latitude, crs='EPSG:4326')
    routes['endp'] = gpd.points_from_xy(valids.dropoff_longitude, valids.dropoff_latitude, crs='EPSG:4326')
    routes['length'] = routes.to_crs('EPSG:32618').geometry.length

#    cur.executemany(insert_sql, [tuple(r.values) for _, r in routes.iterrows()])
    routes.to_postgis('routes', engine, if_exists='append', index=False,
                      dtype={
                          'route': Geometry(geometry_type='LINESTRING', srid=4326),
                          'startp': Geometry(geometry_type='POINT', srid=4326),
                          'endp': Geometry(geometry_type='POINT', srid=4326),
                      }
     )

    print(f'Requests {last}:{new} took {round(time.time()-start, 1)} secs.')
    last = new

print(f'Calculated {len(period)} in {round((time.time()-tot_start)/3660, 2)} hours.')